In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt

import seaborn as sb
import re

Mounted at /content/drive


In [ ]:
import statistics as st
import scipy.stats as ss

In [ ]:

## matrizes das similaridades usando as top-50 palavras dos topicos
for m in range(1,12):
  mat = pd.read_csv('/content/drive/My Drive/datasets/articles/monthPre/final/arquivos meses/modelos/meses/mesesTopicos/matrizes M com M+1/rowMes{}colMes{}.csv'.format(m, m+1))
  mat.drop('Unnamed: 0',inplace=True, axis=1)

In [ ]:
mat = pd.read_csv('/content/drive/My Drive/datasets/articles/monthPre/final/arquivos meses/modelos/meses/mesesTopicos/matrizes M com M+1/rowMes{}colMes{}.csv'.format(m, m+1))
mat.drop('Unnamed: 0',inplace=True, axis=1)

In [ ]:
m= mat.to_numpy()[:,1:]

In [ ]:
maxValues = []
minValues = []

In [ ]:
for m in range(1,12):
  mat = pd.read_csv('/content/drive/My Drive/datasets/articles/monthPre/final/arquivos meses/modelos/meses/mesesTopicos/matrizes M com M+1/rowMes{}colMes{}.csv'.format(m, m+1))
  mat.drop('Unnamed: 0',inplace=True, axis=1)
  m = mat.to_numpy()[:,1:]
  for t in range(30):
    maxValues.append(m[t].max())
    minValues.append(m[t].min())

In [ ]:
arrMax = np.array(maxValues)
arrMin = np.array(minValues)

In [ ]:
print("Menor valor das melhores similaridades entre os topicos", arrMax.min())
print("Media das melhores similaridades entre os topicos", arrMax.mean())
print("Desvio padrão", arrMax.std())

Menor valor das melhores similaridades entre os topicos 0.3173777759075165
Media das melhores similaridades entre os topicos 0.8509182564236901
Desvio padrão 0.16657969895045374


In [ ]:
st.mean(maxValues)

0.8509182564236901

In [ ]:
arrMax.min() + arrMax.std() #abaixo disso topicos morreram 

0.48395747485797025

In [ ]:
print("abaixo de {} topicos morreram, entre {} e  {} tiveram uma mudança simples e maiores do que {} continuam o mesmo ".format((arrMax.min() + arrMax.std()), (arrMax.min() + arrMax.std()), (arrMax.mean()),  (arrMax.mean()) ))

abaixo de 0.48395747485797025 topicos morreram, entre 0.48395747485797025 e  0.8509182564236901 tiveram uma mudança simples e maiores do que 0.8509182564236901 continuam o mesmo 


Topicos abaixo de 0.484 morrem

Topicos entre 0.484 e 0.60 se tiverem outro topico que somado a probabilidade  passa de 0.9 é split

Topicos entra 0.484 e 0.85 que nao splitaram são similares

Topicos maiores que 0.85 são os mesmos ou muito similares

Merge acontece "naturalmente" quando um ou mais topicos anteriores se ligam em um topico posterior 

In [ ]:
def merged_topics(df, month, died, splitd):
  posCounts = df.pos.value_counts()
  ts = posCounts.keys()
  topicsMerg = []
  merged = []
  for t in ts:
    if posCounts[t] > 1:
      topicsMerg.append(t)
  for t in topicsMerg:
    ant = df.ant[df.pos == t]
    
    for at in ant:
      if at not in died and at not in splitd:
        tpc = df.antTopic[df.ant == at]
        print('** topico', at)
        [print(word) for word in tpc.values]
        merged.append(at)

    print("*****MERGED topic", t, 'month', m+1)
    topicPos = df.posTopic[df.pos == t]
    print(topicPos.values[0])
    print('\n')
  return merged



In [ ]:
topics= [str(i) for i in range(30)]

In [ ]:
def split(m, df):
  splitd = []
  mesX = pd.read_csv('/content/drive/My Drive/datasets/articles/monthPre/final/arquivos meses/modelos/meses/mesesTopicos/topicos/topicosMes{}.csv'.format(m))
  mesY = pd.read_csv('/content/drive/My Drive/datasets/articles/monthPre/final/arquivos meses/modelos/meses/mesesTopicos/topicos/topicosMes{}.csv'.format(m+1))

  Mat=pd.read_csv('/content/drive/My Drive/datasets/articles/monthPre/final/arquivos meses/modelos/meses/mesesTopicos/matrizes M com M+1/rowMes{}colMes{}.csv'.format(m, m+1))
  Mat.drop('Unnamed: 0',inplace=True, axis=1)
  mat=Mat.to_numpy()[:,1:]
  for tAt, linha in enumerate(mat):
    maior = 0
    t = 0
    for tPos, col in enumerate(linha):
      if col > maior:
        maior = col
        t = tPos
    
    if maior > 0.48 and maior <= 0.6:
        tSplit = -1
        probSplit = -1
        for tPos, col in enumerate(linha):
          if tPos != t:
            if (col + maior) >= 0.90:
              tSplit = tPos
              probSplit = col;
        if tSplit != -1 and probSplit != -1:
          splitd.append(tAt)
          topicoAtual = mesX.loc[[tAt]].topicosSemProb.values[0]
          topicoSplit = mesY.loc[[tSplit]].topicosSemProb.values[0]
          topicoMaior = mesY.loc[[t]].topicosSemProb.values[0]
          
          #print('{} tSplit {}:{} e {}:{}'.format(tAct, tSplit, probSplit, t, maior))
          print("**topico {} month {} \n {} \n * SPLIT month {} \n *topico {}: {} \n *topico {}: {}\n".format(tAt, m, topicoAtual, m+1, tSplit, topicoSplit, t, topicoMaior))
  return splitd

WRITE TXT


In [ ]:
def merged_topicsw(df, month, f, died, splitd):
  posCounts = df.pos.value_counts()
  ts = posCounts.keys()
  merged = []
  topicsMerg = []
  for t in ts:
    if posCounts[t] > 1:
      topicsMerg.append(t)
  for t in topicsMerg:
    ant = df.ant[df.pos == t] 
    for at in ant:
      if at not in died and at not in splitd:
        tpc = df.antTopic[df.ant == at]
        f.write('** topico {}\n'.format( at+ 1))
        [f.write(word) for word in tpc.values]
        f.write('\n')
        merged.append(at)

    f.write("*****MERGED topic {} month {}\n".format(t+ 1, m+1))
    topicPos = df.posTopic[df.pos == t]
    f.write(topicPos.values[0])
    f.write('\n\n\n')
  return merged

def splitw(m, df, f):
  splitd = []
  mesX = pd.read_csv('/content/drive/My Drive/datasets/articles/monthPre/final/arquivos meses/modelos/meses/mesesTopicos/topicos/topicosMes{}.csv'.format(m))
  mesY = pd.read_csv('/content/drive/My Drive/datasets/articles/monthPre/final/arquivos meses/modelos/meses/mesesTopicos/topicos/topicosMes{}.csv'.format(m+1))

  Mat=pd.read_csv('/content/drive/My Drive/datasets/articles/monthPre/final/arquivos meses/modelos/meses/mesesTopicos/matrizes M com M+1/rowMes{}colMes{}.csv'.format(m, m+1))
  Mat.drop('Unnamed: 0',inplace=True, axis=1)
  mat=Mat.to_numpy()[:,1:]
  for tAt, linha in enumerate(mat):
    maior = 0
    t = 0
    for tPos, col in enumerate(linha):
      if col > maior:
        maior = col
        t = tPos
    
    if maior > 0.48 and maior <= 0.6:
        tSplit = -1
        probSplit = -1
        for tPos, col in enumerate(linha):
          if tPos != t:
            if (col + maior) >= 0.90:
              tSplit = tPos
              probSplit = col;
        if tSplit != -1 and probSplit != -1:
          splitd.append(tAt)
          topicoAtual = ", ".join(w for w in mesX.loc[[tAt]].topicosSemProb.values[0].split(',')[0:11])
          topicoSplit = ", ".join(w for w in mesY.loc[[tSplit]].topicosSemProb.values[0].split(',')[0:11])
          topicoMaior = ", ".join(w for w in mesY.loc[[t]].topicosSemProb.values[0].split(',')[0:11])
          
          #print('{} tSplit {}:{} e {}:{}'.format(tAct, tSplit, probSplit, t, maior))
          f.write("\n**topico {} month {} \n {} \n ****SPLIT month {} \n *topico {}: {} \n *topico {}: {}\n".format(tAt+1, m, topicoAtual, m+1, tSplit+1, topicoSplit, t+1, topicoMaior))
  return splitd

In [ ]:
for m in range(1, 12):
  f  = open(('/content/drive/My Drive/datasets/articles/monthPre/final/arquivos meses/modelos/meses/mesesTopicos/txt/mes{}com{}.txt'.format(m, m+1)), 'w')
  df = pd.read_csv('/content/drive/My Drive/datasets/articles/monthPre/final/arquivos meses/modelos/meses/mesesTopicos/track/M com M+1/mes{}com{}.csv'.format(m, m+1))
  f.write("\n\n ************** MES {} COM {} *******************".format(m, m+1))

  f.write("\n -------------------------DIED/BORN \n\n")
  died = []
  for row in df.itertuples():
    if row.prob <= 0.484: ##DIED
      died.append(row.ant)
      f.write('\n')
      f.write('-> {}\n'.format(round(row.prob, 4)))
      f.write("**DIED M{}: {} - {}\n".format(m, row.ant + 1,row.antTopic,))
      f.write("****BORN M{}: {} - {}\n".format(m+1, row.pos+ 1, row.posTopic))

  f.write('\n ------------------------- SPLIT\n\n')  
  spltd = splitw(m, df, f)

  f.write('\n ------------------------- MERGE \n\n')  
  merged = merged_topicsw(df, m, f, died, spltd)
  
  f.write("\n ------------------------SIMILIAR \n\n")
  for row in df.itertuples():
    if row.ant not in merged and row.ant not in spltd:
      if row.prob > 0.484 and row.prob <= 0.85:
        f.write('\n')
        f.write('-> {}\n'.format(round(row.prob, 4)))
        f.write("**M{} - {} - {}\n".format(m, row.ant+ 1,row.antTopic))
        f.write("****M{} - {} - {}\n".format(m+1, row.pos+ 1,row.posTopic))

  f.write('\n -------------------------SAME \n\n')
  for row in df.itertuples():
    if row.ant not in merged and row.ant not in spltd:
      if row.prob > 0.85:
        f.write('\n')
        f.write('-> {}\n'.format(round(row.prob, 4)))
        f.write("**M{} - {} - {}\n".format(m, row.ant+ 1,row.antTopic))
        f.write("****M{} - {} - {}\n".format(m+1, row.pos+ 1,row.posTopic))
  f.close()


## tracking

In [ ]:
df = pd.read_csv('/content/drive/My Drive/datasets/articles/monthPre/final/arquivos meses/modelos/meses/mesesTopicos/track/M com M+1/mes{}com{}.csv'.format(1, 2))

In [ ]:
df = pd.read_csv('/content/drive/My Drive/datasets/articles/monthPre/final/arquivos meses/modelos/meses/mesesTopicos/track/M com M+1/mes{}com{}.csv'.format(1, 2))

In [ ]:
ant = []
pos = []

In [ ]:
posBirth = []

In [ ]:
posBirth

[15, 28, 8]

In [ ]:
for row in df.itertuples():
  ant.append(row.ant)
  if row.prob <= 0.484:
    pos.append(None)
    posBirth.append(row.pos)
  else:
    pos.append(row.pos)

In [ ]:
def splitTrack(m):
  splitd = {}
  mesX = pd.read_csv('/content/drive/My Drive/datasets/articles/monthPre/final/arquivos meses/modelos/meses/mesesTopicos/topicos/topicosMes{}.csv'.format(m))
  mesY = pd.read_csv('/content/drive/My Drive/datasets/articles/monthPre/final/arquivos meses/modelos/meses/mesesTopicos/topicos/topicosMes{}.csv'.format(m+1))

  Mat=pd.read_csv('/content/drive/My Drive/datasets/articles/monthPre/final/arquivos meses/modelos/meses/mesesTopicos/matrizes M com M+1/rowMes{}colMes{}.csv'.format(m, m+1))
  Mat.drop('Unnamed: 0',inplace=True, axis=1)
  mat=Mat.to_numpy()[:,1:]
  for tAt, linha in enumerate(mat):
    maior = 0
    t = 0
    for tPos, col in enumerate(linha):
      if col > maior:
        maior = col
        t = tPos
    #se a prob esta entre 0.484 e 0.6, então verifica se tem outra probabilidade que somado com a atual passa de 0.90, configurando um split.
    if maior > 0.484 and maior <= 0.6:
        tSplit = -1
        probSplit = -1
        for tPos, col in enumerate(linha):
          if tPos != t:
            if (col + maior) >= 0.90:
              tSplit = tPos
              probSplit = col;
        if tSplit != -1 and probSplit != -1:
          splitd[tAt] = []
          splitd[tAt].append(tSplit+1)
          splitd[tAt].append(t+1)
          topicoAtual = mesX.loc[[tAt]].topicosSemProb.values[0]
          topicoSplit = mesY.loc[[tSplit]].topicosSemProb.values[0]
          topicoMaior = mesY.loc[[t]].topicosSemProb.values[0]
          
          #print('{} tSplit {}:{} e {}:{}'.format(tAct, tSplit, probSplit, t, maior))
          #print("**topico {} month {} \n {} \n * SPLIT month {} \n *topico {}: {} \n *topico {}: {}\n".format(tAt, m, topicoAtual, m+1, tSplit, topicoSplit, t, topicoMaior))
  return splitd

In [ ]:
t = splitTrack(3)

In [ ]:
t

{0: [13, 1], 18: [8, 2]}

In [ ]:
dfDict = {
          'm1': [],
          'm1c2': [],
          'm2': [],
          'm2c3': [],
          'm3': [],
          'm3c4': [],
          'm4': [],
          'm4c5': [],
          'm5': [],
          'm5c6': [],
          'm6': [],
          'm6c7': [],
          'm7': [],
          'm7c8': [],
          'm8': [],
          'm8c9': [],
          'm9': [],
          'm9c10':[],
          'm10': [],
          'm10c11': [],
          'm11': [],
          'm11c12': [],
          }

In [ ]:
posB = []
for m in [1,2,3,4,5,6,7,8,9,10,11]:
  ant = []
  pos =[]
  
  dfm = df = pd.read_csv('/content/drive/My Drive/datasets/articles/monthPre/final/arquivos meses/modelos/meses/mesesTopicos/track/M com M+1/mes{}com{}.csv'.format(m, m+1))

  splt = splitTrack(m)
  for row in df.itertuples():
    if row.ant in splt.keys():
      ant.append(row.ant+1)
      spltStr = ", ".join(str(v) for v in splt[row.ant])
      pos.append(spltStr)
    elif row.prob <= 0.484:
      pos.append(row.pos + 1)
      #pos.append(None)
      ant.append(-1*(row.ant + 1))
    else:
      ant.append(row.ant + 1)
      pos.append(row.pos + 1)
    
  key1 = 'm{}'.format(m)
  key2 = 'm{}c{}'.format(m, m+1)

  for a in ant:
    dfDict[key1].append(a)
  
  for p in pos:
    dfDict[key2].append(p)


In [ ]:
for k in dfDict.keys():
  print(len(dfDict[k]))

In [ ]:
save = pd.DataFrame.from_dict(dfDict)

In [ ]:
save.to_csv("/content/drive/My Drive/datasets/articles/monthPre/final/arquivos meses/modelos/meses/mesesTopicos/testeDiagram.csv")

##tracking 2


In [ ]:
dfDict = {'m1': [], 'm2':[], 'm3': [], 'm4':[], 'm5': [], 'm6': [], 'm7': [], 'm8': [], 'm9': [], 'm10': [], 'm11': [], 'm12': []}

In [ ]:
for topic in range(30):
  t = topic
  
  for m in [1,2,3,4,5,6,7,8,9,10,11]:
    
    dfm = df = pd.read_csv('/content/drive/My Drive/datasets/articles/monthPre/final/arquivos meses/modelos/meses/mesesTopicos/track/M com M+1/mes{}com{}.csv'.format(m, m+1))
    pos = dfm.pos[ dfm.ant == t ]
    ant = dfm.ant[ dfm.ant == t ]
    ant = ant.values[0]
    prob = dfm.prob[ dfm.ant == t]
    prob = prob.values[0]

    if prob <= 0.484:
      dfDict['m{}'.format(m)].append(-1*(ant + 1))
    else:
      dfDict['m{}'.format(m)].append(ant + 1)
    if m == 11:
      dfDict['m12'].append(pos.values[0])
  
    t = pos.values[0]

In [ ]:
for k in dfDict.keys():
  print(len(dfDict[k]))

In [ ]:
dfDict['m2']

In [ ]:
save = pd.DataFrame.from_dict(dfDict)

In [ ]:
save.to_csv("/content/drive/My Drive/datasets/articles/monthPre/final/arquivos meses/modelos/meses/mesesTopicos/vis2.csv")

In [ ]:
save

##tracking 3
